<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Scale_Factor_Evolution_with_Torsion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Plot 1: Scale Factor Evolution with Torsion
# ===========================================

# Constants (illustrative units)
G = 6.674e-11    # m³/(kg·s²)
c = 3e8          # m/s
kappa = 8 * np.pi * G / c**4  # ~1.87e-44 (SI)
sigma = 1e36      # Adjusted to show bounce (kg/m³)
H0 = 2.2e-18     # ~70 km/s/Mpc in s⁻¹
rho0 = kappa * sigma**2  # Initial density

# Friedmann equations
def friedmann_EC(t, a):
    rho = rho0 / a**3  # Matter density
    H_squared = (8 * np.pi * G / 3) * (rho - 0.5 * kappa * sigma**2) - 1/a**2
    return np.sqrt(np.abs(H_squared)) * np.sign(H_squared)

def friedmann_LCDM(t, a):
    rho = rho0 / a**3
    H_squared = (8 * np.pi * G / 3) * rho - 1/a**2
    return np.sqrt(np.abs(H_squared)) * np.sign(H_squared)

# Solve ODEs
t_span = (-1e10, 1e10)  # Adjusted time span for visibility
a0 = 1.0  # Initial scale factor at bounce
sol_EC = solve_ivp(friedmann_EC, t_span, [a0], method='LSODA', dense_output=True)
sol_LCDM = solve_ivp(friedmann_LCDM, t_span, [a0], method='LSODA', dense_output=True)

t_values = np.linspace(-1e10, 1e10, 1000)
a_EC = sol_EC.sol(t_values)[0]
a_LCDM = sol_LCDM.sol(t_values)[0]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(t_values, a_EC, label='EC Theory (Bounce)')
plt.plot(t_values, a_LCDM, '--', label='ΛCDM (Singularity)')
plt.xlabel('Time (s)')
plt.ylabel('Scale Factor $a(t)$')
plt.title('Scale Factor Evolution: EC vs ΛCDM')
plt.legend()
plt.grid(True)
plt.show()

# Plot 2: Spin Density vs. Critical Density
# =========================================

# Parameters (adjusted for transition)
rho0_crit = 9e-27  # Current matter density (kg/m³)
sigma_const = 1e10  # Spin density (kg/m³)
kappa_crit = 1e-45  # Adjusted for visibility

# Redshift range
z = np.linspace(0, 1000, 100)
rho_z = rho0_crit * (1 + z)**3  # Matter density evolution
sigma_crit = np.sqrt(2 * rho_z / kappa_crit)  # Critical spin density

plt.figure(figsize=(10, 6))
plt.loglog(z, sigma_crit, label='Critical Spin Density $\\sigma_{\\rm crit}(z)$')
plt.axhline(y=sigma_const, color='r', linestyle='--', label='Spin Density $\\sigma = 10^{10}$ kg/m³')
plt.fill_between(z, sigma_crit, sigma_crit.max(), where=(sigma_const > sigma_crit),
                 color='green', alpha=0.3, label='Torsion Dominates ($\\sigma > \\sigma_{\\rm crit}$)')
plt.fill_between(z, 0, sigma_crit, where=(sigma_const < sigma_crit),
                 color='orange', alpha=0.3, label='GR Regime ($\\sigma \\ll \\sigma_{\\rm crit}$)')
plt.xlabel('Redshift $z$')
plt.ylabel('Density (kg/m³)')
plt.title('Spin Density vs. Critical Density')
plt.legend()
plt.grid(True)
plt.show()